In [224]:
import pandas as pd

In [225]:
df = pd.read_csv("data_uncertain.csv")
print(df)

   TransactionID       Items  Probabilities    Quantity
0              1     A, C, D           0.95     1, 3, 4
1              2     B, C, D           0.80     1, 1, 2
2              3     A, B, E           0.50     2, 2, 3
3              4        C, E           0.95        2, 2
4              5     B, D, E           0.70     1, 2, 2
5              6     A, C, D           1.00     1, 2, 1
6              7  A, C, D, E           0.80  3, 1, 3, 4
7              8     B, C, E           0.76     1, 4, 1
8              9        B, D           0.60        3, 5
9             10        D, E           0.90        5, 2


In [226]:
# Chuyển đổi các cột thành danh sách
df['Items'] = df['Items'].apply(lambda x: x.split(', '))
# df['Probabilities'] = df['Probabilities'].apply(lambda x: list(map(float, x.split(', '))))
# df['Utilities'] = df['Utilities'].apply(lambda x: list(map(float, x.split(', '))))
df['Quantity'] = df['Quantity'].apply(lambda x: list(map(float, x.split(', '))))
# Kiểm tra lại dữ liệu
# print(df.head())
print(df)


   TransactionID         Items  Probabilities              Quantity
0              1     [A, C, D]           0.95       [1.0, 3.0, 4.0]
1              2     [B, C, D]           0.80       [1.0, 1.0, 2.0]
2              3     [A, B, E]           0.50       [2.0, 2.0, 3.0]
3              4        [C, E]           0.95            [2.0, 2.0]
4              5     [B, D, E]           0.70       [1.0, 2.0, 2.0]
5              6     [A, C, D]           1.00       [1.0, 2.0, 1.0]
6              7  [A, C, D, E]           0.80  [3.0, 1.0, 3.0, 4.0]
7              8     [B, C, E]           0.76       [1.0, 4.0, 1.0]
8              9        [B, D]           0.60            [3.0, 5.0]
9             10        [D, E]           0.90            [5.0, 2.0]


In [227]:
print(df)

   TransactionID         Items  Probabilities              Quantity
0              1     [A, C, D]           0.95       [1.0, 3.0, 4.0]
1              2     [B, C, D]           0.80       [1.0, 1.0, 2.0]
2              3     [A, B, E]           0.50       [2.0, 2.0, 3.0]
3              4        [C, E]           0.95            [2.0, 2.0]
4              5     [B, D, E]           0.70       [1.0, 2.0, 2.0]
5              6     [A, C, D]           1.00       [1.0, 2.0, 1.0]
6              7  [A, C, D, E]           0.80  [3.0, 1.0, 3.0, 4.0]
7              8     [B, C, E]           0.76       [1.0, 4.0, 1.0]
8              9        [B, D]           0.60            [3.0, 5.0]
9             10        [D, E]           0.90            [5.0, 2.0]


In [228]:
profit = {'A': 6, 'B':3,'C':10,'D':1,'E':5}
items = set()
for l in df['Items']:
    for i in l:
        items.add(i)
items = sorted(items)
TUET = {tid:0 for tid in df['TransactionID']}
for i,l in enumerate(df['TransactionID']):
    for j,o in enumerate(df['Items'][i]):
        TUET[l] += df['Quantity'][i][j]*profit[o]
TU = sum([i for _,i in TUET.items()])
D = len(df['TransactionID'])
print(TUET)

{1: 40.0, 2: 15.0, 3: 33.0, 4: 30.0, 5: 15.0, 6: 27.0, 7: 51.0, 8: 48.0, 9: 14.0, 10: 15.0}


In [229]:

def GetHTWPUI(e,u):
    HTWPUI = {}
    twu = {}
    pro = {}
    for item in items:
        twu = {i:0 for i in items}
        pro = {i:0 for i in items}
    for item in items:
        for i,l in enumerate(df['Items']):
            for j,o in enumerate(l):
                if item == o:
                    # pro[o] += df['Probabilities'][i][j]
                    pro[o] += df['Probabilities'][i]
                    twu[o] += TUET[df['TransactionID'][i]]
                    # twu[o] += df['Probabilities'][i][j]*df['Quantity'][i][j]*profit[o]
                    # twu[o] += df['Probabilities'][i]*df['Quantity'][i][j]*profit[o]
    pro = {k:round(v,2) for k,v in pro.items()}
    twu = {k:round(v,2) for k,v in twu.items()}
    HTWPUI = {k:(twu[k], pro[k]) for k in items if twu[k] >= e*TU and pro[k] >= u*D}
    HTWPUI = dict(sorted(HTWPUI.items(), key=lambda item: item[1][0]))
    return HTWPUI
print(GetHTWPUI(0,0))
# print(GetHTWPUI(0,0)['B'][0])
            

{'B': (125.0, 3.36), 'A': (151.0, 3.25), 'D': (177.0, 5.75), 'E': (192.0, 4.61), 'C': (211.0, 5.26)}


In [230]:
results = []
HTWPUI = GetHTWPUI(0,0)
items = list(HTWPUI.keys())
T = TUET.copy()
for _, row in df.iterrows():
    tid = row['TransactionID']
    items = row['Items']
    quantities = row['Quantity']
    quant = {items[i]:quantities[i] for i in range(len(items)) }
    items = sorted(items, key= lambda x: HTWPUI[x][0])
    probs = row['Probabilities']
    # utilities = [quantities[i] * profit[items[i]] for i in range(len(items))]
    utilities = [quant[items[i]] * profit[items[i]] for i in range(len(items))]
    for i in range(len(items)):
        item = items[i]
        # prob = probs[i]
        iu = utilities[i]
        # ru = sum(utilities[i+1:])
        ru = T[tid] - iu
        T[tid] = ru
        results.append({
            'TransactionID': tid,
            'Item': item,
            'Probability': probs,
            'ItemUtility (iu)': iu,
            'RemainingUtility (ru)': ru
        })

# Chuyển kết quả thành DataFrame để dễ dàng theo dõi
result_df = pd.DataFrame(results)

# In kết quả
print(result_df)

    TransactionID Item  Probability  ItemUtility (iu)  RemainingUtility (ru)
0               1    A         0.95               6.0                   34.0
1               1    D         0.95               4.0                   30.0
2               1    C         0.95              30.0                    0.0
3               2    B         0.80               3.0                   12.0
4               2    D         0.80               2.0                   10.0
5               2    C         0.80              10.0                    0.0
6               3    B         0.50               6.0                   27.0
7               3    A         0.50              12.0                   15.0
8               3    E         0.50              15.0                    0.0
9               4    E         0.95              10.0                   20.0
10              4    C         0.95              20.0                    0.0
11              5    B         0.70               3.0                   12.0

In [231]:
class Pu_List:
    def __init__(self, item):
        self.item = item
        self.entries = []
    def add_entries(self, tid, probability, utility):
        self.entries.append({'tid': tid, 'probability': probability, 'utility':utility,'re_utility': utility})

In [232]:
def pu_list_construction(X, Xa, Xb):
    Xab_PU_list = Pu_List(f"{Xa.item}, {Xb.item}")
    for Ea in Xa.entries:
        matching_Eb = next((Eb for Eb in Xb.entries if Eb['tid'] == Ea['tid']), None)
        if matching_Eb is not None:  
            if X.entries:  
                matching_E = next((E for E in X.entries if E['tid'] == Ea['tid']), None)
                Eab = {
                    'tid': Ea['tid'],
                    'probability': Ea['probability'],
                    'utility': Ea['utility'] + matching_Eb['utility'] - matching_E['utility'],
                    're_utility': matching_Eb['re_utility']
                }
            else:
                Eab = {
                    'tid': Ea['tid'],
                    'probability': Ea['probability'],
                    'utility': Ea['utility'] + matching_Eb['utility'],
                    're_utility': matching_Eb['re_utility']
                }
            Xab_PU_list.add_entries(Eab['tid'], Eab['probability'], Eab['utility'], Eab['re_utility'])
    
    return Xab_PU_list
